In [ ]:
import joblib
import click
import json
import time
import os
import itertools
import collections.abc
from shutil import copy,SameFileError
from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
# from qiskit_ibm_provider import IBMProvider

from quantum.Quantum import QuantumRegressor
from quantum.Evaluate import evaluate
from settings import ANSATZ_LIST, ENCODER_LIST
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import InconsistentVersionWarning

from qiskit.circuit.library import n_local
from qiskit.circuit import QuantumCircuit

In [ ]:
num_qubits=5

wires=range(num_qubits)

# device='lightning.qubit'
device='qulacs.simulator'

device = qml.device(device, wires=num_qubits)


In [ ]:
class TwoLocal:

    def __init__(self,
                 wires: list = None,
                 entanglement: str = 'linear',
                 reps: int = 1,
                 rot_gates: list = None,
                 entangle_gates: list = None,
                 skip_final_rot: bool = True
                 ):
        self._entanglement = entanglement
        self._reps = reps
        self._rot_gates = rot_gates
        self._entangle_gates = entangle_gates
        self._skip_final_rot = skip_final_rot
        self._wires = None
        self._qc = None
        if wires is not None:
            self.set_wires(wires)

    def __call__(self, parameters, wires: list = None):
        if wires is not None:
            if len(self._wires) != len(self._wires):
                raise ValueError("Cannot override wires instance of different length")
            self._wires = wires
        qc = self._qc
        if qc.num_parameters_settable != len(parameters):
            raise ValueError("Incorrect number of parameters. Expected ", qc.num_parameters_settable, "but received ",
                             len(parameters))
        qc = qc.decompose()
        parameters = parameters.tolist()
        qc = qc.assign_parameters(parameters)
        qml_circuit = qml.from_qiskit(qc)
        qml_circuit(wires=self._wires)

    @property
    def num_params(self):
        return self._qc.num_parameters_settable

    @property
    def layers(self):
        return self._reps

    @layers.setter
    def layers(self, val):
        self._reps = val

    def set_wires(self, wires):
        self._wires = wires
        if self._entanglement == 'complete':
            entanglement = []
            for i in wires:
                for j in wires:
                    if i != j:
                        entanglement.append((i, j))
            self._entanglement = entanglement
        self._qc = n_local.TwoLocal(num_qubits=len(self._wires), entanglement=self._entanglement, reps=self._reps,
                                    rotation_blocks=self._rot_gates, entanglement_blocks=self._entangle_gates,
                                    skip_final_rotation_layer=self._skip_final_rot)

In [ ]:
variational = TwoLocal(wires=wires,rot_gates=['rx', 'rz'], entangle_gates=['crx'], entanglement='complete')
LAYERS=1
re_upload_depth=1

variational.layers = LAYERS
# variational.set_wires(wires)

num_params = variational.num_params * re_upload_depth
generator = np.random.default_rng(12958234)
initial_parameters = generator.uniform(-np.pi, np.pi, num_params)
feat = generator.uniform(-1, 1, num_qubits)

def circuit(parameters, wires):
    variational(parameters=parameters, wires=wires)
    return  qml.expval(qml.PauliZ(0))
variational(initial_parameters, wires)
qnode = qml.QNode(circuit,device)
qml.draw_mpl(qnode)(initial_parameters, wires)

In [ ]:


qc = n_local.EfficientSU2(num_qubits=len(wires),su2_gates=['ry', 'rz'],entanglement='linear',reps=1,skip_final_rotation_layer=True)
# print(qc.num_parameters_settable)
params = qc.num_parameters_settable
qc = qc.decompose()

generator = np.random.default_rng(12958234)
initial_parameters = generator.uniform(-np.pi, np.pi, params)
qc = qc.assign_parameters(initial_parameters)
qml_circuit = qml.from_qiskit(qc)

my_qfunc = qml.from_qiskit(qc)


# @qml.qnode(device)
def circuit(wires):
    my_qfunc(wires=wires)
    return qml.expval(qml.Z(0))

qml.draw_mpl(qml.QNode(circuit,device))(wires)

In [ ]:
class EfficientSU2:
    def __init__(self,
                 wires: list = None,
                 su2_gates: list = None,
                 entanglement: str = 'linear',
                 reps: int = 1,
                 skip_final_rot: bool = False
                 ):
        if su2_gates is None:
            su2_gates = ['ry', 'rz']
        self._su2_gates = su2_gates
        self._entanglement = entanglement
        self._reps = reps
        self._skip_final_rot = skip_final_rot
        self._wires = None
        self._qc = None
        if wires is not None:
            self.set_wires(wires)
            
    
    def __call__(self, parameters, wires: list = None):
        if wires is not None:
            
            if len(self._wires) != len(self._wires):
                raise ValueError("Cannot override wires instance of different length")
            self._wires = wires
            
        qc = self._qc
        if qc.num_parameters_settable != len(parameters):
            raise ValueError("Incorrect number of parameters. Expected ", qc.num_parameters_settable, 'but received ',
                             len(parameters))
        qc = qc.decompose()
        parameters = parameters.tolist()
        qc = qc.assign_parameters(parameters)
        qml_circuit = qml.from_qiskit(qc)
        return qml_circuit
        # qml_circuit(wires=self._wires)
        # qml.draw_mpl(qml_circuit)()
        
    @property
    def num_params(self):
        return self._qc.num_parameters_settable

    @property
    def layers(self):
        return self._reps

    @layers.setter
    def layers(self, val):
        self._reps = val
    
    def set_wires(self, wires):
        self._wires = wires
        self._qc = n_local.EfficientSU2(num_qubits=len(self._wires), su2_gates=self._su2_gates,
                                        entanglement=self._entanglement, reps=self._reps,
                                        skip_final_rotation_layer=self._skip_final_rot)


In [ ]:
variational=EfficientSU2(skip_final_rot=True)
LAYERS=1
re_upload_depth=1

variational.layers = LAYERS
variational.set_wires(wires)

num_params = variational.num_params * re_upload_depth
generator = np.random.default_rng(12958234)
initial_parameters = generator.uniform(-np.pi, np.pi, num_params)
feat = generator.uniform(-1, 1, num_qubits)

def circuit(parameters, wires):
    variational(parameters=parameters, wires=wires)
    return  qml.expval(qml.PauliZ(0))
variational(initial_parameters, wires)
qnode = qml.QNode(circuit,device)
qml.draw_mpl(qnode)(initial_parameters, wires)

In [ ]:
import pennylane as qml
import numpy as np

# Ensure `wires` is a list of integers
wires = list(range(num_qubits))  # Define your `num_qubits` explicitly here

# Define your circuit function for QNode
def circuit(parameters):
    variational(parameters)  # `variational` uses `wires` internally
    return qml.expval(qml.PauliZ(0))

# Initialize the PennyLane device with correct wire count
device = qml.device("default.qubit", wires=len(wires))
qnode = qml.QNode(circuit, device)

# Execute the circuit in a loop
for i in range(2):
    try:
        print("Attempting execution")
        result = qnode(parameters=initial_parameters)
    except Exception as e:
        print("Error encountered:", e)
        result = qnode(parameters=initial_parameters)
    print("Result:", result)


In [ ]:
import pennylane as qml
import numpy as np
# from quantum.circuits.Ansatz import TwoLocal
class TwoLocal:

    def __init__(self,
                 wires: list = None,
                 entanglement: str = 'linear',
                 reps: int = 1,
                 rot_gates: list = None,
                 entangle_gates: list = None,
                 skip_final_rot: bool = True
                 ):
        self._entanglement = entanglement
        self._reps = reps
        self._rot_gates = rot_gates
        self._entangle_gates = entangle_gates
        self._skip_final_rot = skip_final_rot
        self._wires = None
        self._qc = None
        if wires is not None:
            self.set_wires(wires)

    def __call__(self, parameters, wires: list = None):
        if wires is not None:
            if len(self._wires) != len(self._wires):
                raise ValueError("Cannot override wires instance of different length")
            self._wires = wires
        qc = self._qc
        if qc.num_parameters_settable != len(parameters):
            raise ValueError("Incorrect number of parameters. Expected ", qc.num_parameters_settable, "but received ",
                             len(parameters))
        qc = qc.decompose()
        parameters = parameters.tolist()
        qc = qc.assign_parameters(parameters)
        qml_circuit = qml.from_qiskit(qc)
        qml_circuit(wires=self._wires)

    @property
    def num_params(self):
        return self._qc.num_parameters_settable

    @property
    def layers(self):
        return self._reps

    @layers.setter
    def layers(self, val):
        self._reps = val

    def set_wires(self, wires):
        self._wires = wires
        if self._entanglement == 'complete':
            entanglement = []
            for i in wires:
                for j in wires:
                    if i != j:
                        entanglement.append((i, j))
            self._entanglement = entanglement
        self._qc = n_local.TwoLocal(num_qubits=len(self._wires), entanglement=self._entanglement, reps=self._reps,
                                    rotation_blocks=self._rot_gates, entanglement_blocks=self._entangle_gates,
                                    skip_final_rotation_layer=self._skip_final_rot)


# Define wires and initialize `EfficientSU2`
num_qubits = 5
wires = list(range(num_qubits))
variational = TwoLocal(wires=wires,rot_gates=['rx', 'rz'], entangle_gates=['crx'], entanglement='complete')
variational.reps = 1

# Generate initial parameters
num_params = variational.num_params
generator = np.random.default_rng(12958234)
initial_parameters = generator.uniform(-np.pi, np.pi, num_params)

# Define the PennyLane circuit function
def circuit(parameters):
    variational(parameters)
    return qml.expval(qml.PauliZ(0))

# Initialize the device and QNode
device = qml.device("qulacs.simulator", wires=wires)
qnode = qml.QNode(circuit, device)

# Run the circuit
for i in range(2):
    try:
        print("Attempting execution")
        result = qnode(parameters=initial_parameters)
    except Exception as e:
        print("Error encountered:", e)
        result = qnode(parameters=initial_parameters)
    print("Result:", result)


In [ ]:
qml.draw_mpl(qnode)(parameters=initial_parameters)